### Node.js Installation
We require the ability to run JavaScript from this notebook.

**Install Node.js dependencies inside of this JupyterLab Docker container**

In [1]:
!mamba install --yes make cxx-compiler nodejs=14.18.3


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.22.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['make', 'cxx-compiler', 'nodejs=14.18.3']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━━╸━━━━━━━━━

**Install Node.js Jupyter kernel inside of this JupyterLab Docker container**

In [2]:
!npm install -g ijavascript
!ijsinstall

npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
/opt/conda/bin/ijs -> /opt/conda/lib/node_modules/ijavascript/bin/ijavascript.js action:finalizeK
/opt/conda/bin/ijsconsole -> /opt/conda/lib/node_modules/ijavascript/bin/ijsconsole.js
/opt/conda/bin/ijsinstall -> /opt/conda/lib/node_modules/ijavascript/bin/ijsinstall.js
/opt/conda/bin/ijskernel -> /opt/conda/lib/node_modules/ijavascript/lib/kernel.js
/opt/conda/bin/ijsnotebook -> /opt/conda/lib/node_modules/ijavascript/bin/ijsnotebook.js

> zeromq@5.2.8 install /opt/conda/lib/node_modules/ijavascript/node_modules/zeromq
> node-gyp-build || npm run build:libzmq

+ ijavascript@5.2.1
added 8 packages from 31 contributors in 5.408s


**Create Node.js project in the current directory and install Fabric Gateway dependencies** \
NOTE: We're using the [legacy SDK](https://github.com/hyperledger/fabric-sdk-node/tree/v2.2.11) below. To do: Upgrade to [Gateway](https://github.com/hyperledger/fabric-gateway/releases/tag/v1.0.1)

In [3]:
!npm init -y
!npm install fabric-ca-client@2.2.11 fabric-network@2.2.11 js-yaml@3.14.1

Wrote to /home/jovyan/work/notebook/package.json:

{
  "name": "notebook",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}


npm WARN deprecated querystring@0.2.0: The querystring API is considered Legacy. new code should use the URLSearchParams API instead.
        ......] | build:yargs: sill linkStuff yargs@16.2.0 has /home/jovyaK0m
> pkcs11js@1.3.0 install /home/jovyan/work/notebook/node_modules/pkcs11js
> node-gyp rebuild

make: Entering directory '/home/jovyan/work/notebook/node_modules/pkcs11js/build'
  CXX(target) Release/obj.target/pkcs11/src/main.o
In file included from ../../nan/nan.h:58,
                 from ../src/main.cpp:1:
/home/jovyan/.cache/node-gyp/14.18.3/include/node/node.h:787:7: warning: cast between incompatible function types from 'void (*)(Nan::ADDON_REGISTER_FUNCTION_ARGS_TYPE)' {aka 'void (*)(v8::Local<v8::Obje

### User Registration

**The CA admin first registers a user (on the Docker host):** \
docker exec -it ca-client.fabric.localhost ./registerUser.js --profile ../connection-profiles/connection-profile-orga.yaml --org orgA --user janssen

### Switch to JavaScript (Node.js) kernel!

**Import modules**

In [1]:
const { Wallets, x509Identity, Gateway } = require('fabric-network');
const FabricCAServices = require('fabric-ca-client');
const yaml = require('js-yaml');
const fs = require('fs');
const path = require('path');

**Configure the user (using the credentials provided by the organization's CA)**

In [2]:
const org = 'orgA';
const user = 'frank';
const secret = 'koZsGJABjRau';
const connectionProfile = yaml.safeLoad(fs.readFileSync(path.resolve(__dirname, '../connection-profiles/connection-profile-orga.yaml'),'utf8'));

### CA Enrollment

In [3]:
async function enroll() {
  // Set up the CA connection
  var certAuth = connectionProfile.organizations[org].certificateAuthorities[0];
  const caInfo = connectionProfile.certificateAuthorities[certAuth];
  const caTLSCACerts = caInfo.tlsCACerts.pem;
  const ca = new FabricCAServices(caInfo.url, { trustedRoots: caTLSCACerts, verify: false }, caInfo.caName);
    
  // Enroll the user
  const enrollment = await ca.enroll({ enrollmentID: user, enrollmentSecret: secret });
    
  // Save the identity in the wallet
  const mspId = connectionProfile.organizations[org].mspid;
  const identity = {
    credentials: {
      certificate: enrollment.certificate,
      privateKey: enrollment.key.toBytes(),
    },
    mspId: mspId,
    type: 'X.509',
  };
  const wallet = await Wallets.newFileSystemWallet('./wallet');
  wallet.put(user, identity);
    
  console.log('Enrollment complete!');
}
enroll();

Promise { <pending> }

Enrollment complete!


### Gateway / Scenarios

In [4]:
async function gateway() {
  const wallet = await Wallets.newFileSystemWallet('./wallet')
  const gateway = new Gateway();
  try { 
    await gateway.connect(connectionProfile, {
      wallet,
      identity: user,
      discovery: {
        enabled: true,
        asLocalhost: false
      }
    });
    const network = await gateway.getNetwork('consortium-chain');
    const contract = network.getContract('consortium-cc-ipfs');
    
    // Submit transactions for the smart contract
    const submitResult = await contract.submitTransaction('addNetwork', 'net1', "dns1;dns2", "key", "acl");

    console.log('Transaction complete!');
  } catch(error) {
    console.log(error);
  } finally {
    gateway.disconnect();
  }
}
gateway();

Promise { <pending> }

2022-04-09T17:45:06.154Z - error: [Transaction]: Error: No valid responses from any peers. Errors:
    peer=peer1.orga.fabric.localhost, status=500, message=Only admins can add IPFS network descriptions.
    peer=peer0.orgc.fabric.localhost, status=500, message=Only admins can add IPFS network descriptions.
    at newEndorsementError (/home/jovyan/work/notebook/node_modules/fabric-network/lib/transaction.js:74:12)
    at getResponsePayload (/home/jovyan/work/notebook/node_modules/fabric-network/lib/transaction.js:41:23)
    at Transaction.submit (/home/jovyan/work/notebook/node_modules/fabric-network/lib/transaction.js:255:28)
    at processTicksAndRejections (internal/process/task_queues.js:95:5)
    at async gateway (evalmachine.<anonymous>:17:26)


Error: No valid responses from any peers. Errors:
    peer=peer1.orga.fabric.localhost, status=500, message=Only admins can add IPFS network descriptions.
    peer=peer0.orgc.fabric.localhost, status=500, message=Only admins can add IPFS network descriptions.
    at newEndorsementError (/home/jovyan/work/notebook/node_modules/fabric-network/lib/transaction.js:74:12)
    at getResponsePayload (/home/jovyan/work/notebook/node_modules/fabric-network/lib/transaction.js:41:23)
    at Transaction.submit (/home/jovyan/work/notebook/node_modules/fabric-network/lib/transaction.js:255:28)
    at processTicksAndRejections (internal/process/task_queues.js:95:5)
    at async gateway (evalmachine.<anonymous>:17:26) {
  responses: [
    {
      version: 0,
      timestamp: null,
      response: [Object],
      payload: <Buffer 0a 20 4b 33 dc 31 18 1c 30 d0 22 4b 27 e9 54 89 72 d9 72 29 26 b3 53 6c 53 f8 03 0d 7b 45 3a f0 e0 83 12 99 01 0a 45 12 43 0a 0a 5f 6c 69 66 65 63 79 ... 140 more bytes>,
     